In [ ]:
!pip install trino

In [ ]:
import configparser
import os
from trino.dbapi import connect
from contextlib import closing
import urllib3
import pandas as pd

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

###
# This script is not meant for automation.
# It is a collection of sql ETL pipeline commands that were used to create and query our test data.
# There is no ordering guarantees and no guarantees of reproducibility.
##


config = configparser.ConfigParser()
config.read('config.ini')

trinoHost=os.getenv('TRINO_HOST', config.get('default', option='host', fallback='localhost'))
trinoPort=os.getenv('TRINO_PORT', int(config.get('default', option='port', fallback=8080)))
trinoScheme=config.get('default', option='scheme', fallback='https')
catalog='s3'

print(f"{trinoScheme}://{trinoHost}:{trinoPort}/")
conn = connect(
    host=trinoHost,
    port=trinoPort,
    user='test',
    http_scheme=trinoScheme,
    verify=False
)

In [ ]:
aggregatePercentiles = f"""
WITH
        stop  AS ( SELECT TRY(CAST(substr(stop_id,1,7) as INTEGER)) as bpuic, stop_lat, stop_lon FROM iceberg.com490_ice.sbb_stops_parquet_part WHERE year=2024 AND month=9 AND day=9),
        shape AS ( SELECT ST_GeomFromBinary(wkb_geometry) as geometry, name FROM iceberg.com490_ice.geo_parquet WHERE level='city' ),
        geo_tagged_stop AS ( SELECT stop.bpuic, stop.stop_lat, stop.stop_lon, shape.name FROM stop JOIN shape ON ST_Contains(shape.geometry, ST_Point(stop.stop_lon, stop.stop_lat))),
        geo_tagged_istdaten AS (
                SELECT day_of_week(istdaten.arr_actual) as day_week, hour(istdaten.arr_actual), (istdaten.arr_actual - istdaten.arr_time) as arr_delay, (istdaten.dep_actual - istdaten.dep_time) as dep_delay, geo_tagged_stop.name
                FROM iceberg.com490_ice.sbb_istdaten_parquet_part AS istdaten JOIN geo_tagged_stop USING (bpuic)
        )
SELECT AVG(arr_delay) as arr_delay, AVG(dep_delay) as dep_delay, COUNT(*) as num, approx_percentile(to_milliseconds(arr_delay)/1000, ARRAY[0.25,0.5,0.75]), name FROM geo_tagged_istdaten GROUP BY name ORDER BY name
"""

In [ ]:
aggregatePercentiles = f"""

WITH
    stop  AS ( SELECT TRY(CAST(substr(stop_id,1,7) as INTEGER)) as bpuic, stop_lat, stop_lon FROM iceberg.com490_ice.sbb_stops_parquet_part WHERE year=2024 AND month=9 AND day=9),
    shape AS ( SELECT ST_GeomFromBinary(wkb_geometry) as geometry, name FROM iceberg.com490_ice.geo_parquet WHERE level='city' ),
    geo_tagged_stop AS ( SELECT stop.bpuic, stop.stop_lat, stop.stop_lon, shape.name FROM stop JOIN shape ON ST_Contains(shape.geometry, ST_Point(stop.stop_lon, stop.stop_lat))),
    geo_tagged_istdaten AS (
        SELECT day_of_week(istdaten.arr_actual) as day_week, hour(istdaten.arr_actual) hour_day, (istdaten.arr_actual - istdaten.arr_time) as arr_delay, (istdaten.dep_actual - istdaten.dep_time) as dep_delay, geo_tagged_stop.name
        FROM iceberg.com490_ice.sbb_istdaten_parquet_part AS istdaten JOIN geo_tagged_stop USING (bpuic)
    )
-- SELECT COUNT(), name FROM geo_tagged_istdaten GROUP BY name
SELECT AVG(arr_delay) as arr_delay, AVG(dep_delay) as dep_delay, COUNT() as num, approx_percentile(to_milliseconds(arr_delay)/1000, ARRAY[0.25,0.5,0.75]), hour_day, name FROM geo_tagged_istdaten WHERE day_week>=1 AND day_week <= 5  GROUP BY name,hour_day ORDER BY name,hour_day
"""

In [ ]:
with closing(conn.cursor()) as cur:
    cur.execute(aggregatePercentiles)
    rows = cur.fetchall()
    for row in rows:
       print(f">>>>> {row}")


# columns = ['Time1', 'Time2', 'Value', 'List', 'Location']
# df = pd.DataFrame(rows, columns=columns)
# print(df.head())

In [ ]:
columns = ['Time1', 'Time2', 'Value', 'List', "Hour", 'Location']
df = pd.DataFrame(rows, columns=columns)
print(df.head())

In [ ]:
df["Quantile_middle"] = df["List"].apply(lambda x: x[1])
print(df.head())
print(df["Quantile_middle"].mean())
print(df["Quantile_middle"].min())
print(df["Quantile_middle"].max())

print(df["Location"].unique())

In [ ]:
import geopandas as gpd
gdf = gpd.read_file('./swissboundaries3d_2024-01_2056_5728.shp/swissBOUNDARIES3D_1_5_TLM_HOHEITSGEBIET.shp')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.express as px

def get_color(value):
    value2 = int(value)
    if (value2 == -9999): return "#EEEEEE"
    
    if value2 < 0:
        # Normalize the value to the range [0, 1] for the green gradient
        norm_value = (value2 + 100) / 100  # -100 maps to 0, 0 maps to 1
        color = plt.cm.Greens(norm_value)
    else:
        # Normalize the value to the range [0, 1] for the red gradient
        norm_value = value2 / 300  # 0 maps to 0, 300 maps to 1
        color = plt.cm.Reds(norm_value)
    
    # Convert the RGBA color to a hex string
    return mcolors.to_hex(color)

# merged_gdf = gdf.merge(df, left_on="NAME", right_on="Location")
# merged_gdf["color"] = merged_gdf["Quantile_middle"].map(get_color)

# fig = px.choropleth(
#     merged_gdf,
#     geojson=merged_gdf.geometry,
#     locations=merged_gdf.index,
#     color='color',
#     hover_name='Location',
#     animation_frame='Hour',
#     projection='mercator',
#     title='Map of Switzerland - 24 Hour Animation'
# )
os.makedirs('./saved_fig', exist_ok=True)

for i in range(0, 24):
    df2 = df[df["Hour"] == i]
    merged_gdf = gdf.merge(df2, left_on="NAME", right_on="Location", how="left")
    merged_gdf["Quantile_middle"].fillna(-9999, inplace=True)
    merged_gdf["Hour"].fillna(i, inplace=True)
    merged_gdf["Location"].fillna(merged_gdf["NAME"], inplace=True)

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    merged_gdf["color"] = merged_gdf["Quantile_middle"].map(get_color)
    merged_gdf.plot(column='Location', ax=ax, legend=True, color=merged_gdf["color"])

    plt.title('Map of Switzerland at hour ' + str(i))
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    # Show the plot
    # Save the plot
    plt.savefig(f'./saved_fig/swiss_{i}.png')

    # Close the plot to free memory
    plt.close(fig)

In [ ]:
from PIL import Image

# List of image file paths
images = ['./saved_fig/swiss_0.png', 
          './saved_fig/swiss_1.png', 
          './saved_fig/swiss_2.png', 
          './saved_fig/swiss_3.png',
          './saved_fig/swiss_4.png',
          './saved_fig/swiss_5.png',
          './saved_fig/swiss_6.png',
          './saved_fig/swiss_7.png',
          './saved_fig/swiss_8.png',
          './saved_fig/swiss_9.png',
          './saved_fig/swiss_10.png',
          './saved_fig/swiss_11.png',
          './saved_fig/swiss_12.png',
          './saved_fig/swiss_13.png',
          './saved_fig/swiss_14.png',
          './saved_fig/swiss_15.png',
          './saved_fig/swiss_16.png',
          './saved_fig/swiss_17.png',
          './saved_fig/swiss_18.png',
          './saved_fig/swiss_19.png',
          './saved_fig/swiss_20.png',
          './saved_fig/swiss_21.png',
          './saved_fig/swiss_22.png',
          './saved_fig/swiss_23.png'
        ]


# Open images and store them in a list
frames = [Image.open(image) for image in images]

# Save frames as an animated GIF
frames[0].save(
    'animated.gif',
    save_all=True,
    append_images=frames[1:],
    duration=300,
    loop=0
)

